# **LAB 04 - Lập trình Spark với Python**

**Thông tin các thành viên**

MSSV | Họ tên 
---|---
Đỗ Nhật Toàn | 19120688 
Lê Quốc Trí | 19120691 
Đào Xuân Tùng | 19120707 
Đinh Nhật Tường | 19120709 

# Khởi tạo Spark

In [ ]:
!python --version

Python 3.7.13


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz

In [ ]:
!tar xzf spark-3.2.1-bin-hadoop3.2.tgz
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 27 kB/s 
     |████████████████████████████████| 199 kB 50.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=a9ac058ed017d9f2ee38e22a3b96904600f8e5cc60e80641d992ffbe0cc1b226
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
# dataset link: https://drive.google.com/file/d/1qakfZ5h7wJzZpjdGzs7Nv0kHeMGFv-RV/view

Py4JError: ignored

In [ ]:
!wget "https://drive.google.com/uc?export=download&id=1qakfZ5h7wJzZpjdGzs7Nv0kHeMGFv-RV" -O "Lab04-Data.zip"
!unzip "/content/Lab04-Data.zip" 

# **Bài 1 - Khai thác mẫu phổ biến và luật kết hợp**

**1) Đọc dữ liệu vào PySpark và tiền xử lý dữ liệu**

In [ ]:
order_df = spark.read.csv("Lab04-Data/orders.csv", header='True')
products_df = spark.read.csv("Lab04-Data/products.csv", header='True')

In [ ]:
order_df.printSchema()
products_df.printSchema()

In [ ]:
order_df.show(5)
products_df.show(5)

- Để thực một số phương thức tiền xử lý dữ liệu, ta sử dụng các fuctions của PySpark SQL.
- Để có thể lấy được tên các sản phẩm (products) của từng mã order ta thực hiện phép join 2 bảng có điều kiện, sử dụng <em>GroupBy</em> để thao thác trên từng order id, ứng với mỗi order id ta sử dụng phương thức <em>collect_list</em> trên thuộc tính products để tạo ra danh sách các sản phẩm.

In [ ]:
from pyspark.sql import functions as f

In [ ]:
df = order_df.join(products_df,order_df['product_id']==products_df['product_id'])\
      .groupBy('order_id')\
      .agg(f.collect_list('product_name').alias('products'))\
      .orderBy('order_id')\
      .select('order_id','products')

In [ ]:
df.show()

In [ ]:
df.count()

**2) Áp dụng giải thuật khai thác mẫu phổ biến và luật kết hợp trong gói pyspark.ml.fpm**

Khởi tạo mô hình FPGrowth của PySpark qua thư viện <em>pyspark.ml.fpm</em> và chạy trên tập dữ liệu đã tiền xử lý

In [ ]:
import pyspark.ml.fpm as fpm

Khởi tạo FPGrowth với:
- minSupport = 0.01
- minConfidence = 0.01

In [ ]:
fpg = fpm.FPGrowth(itemsCol='products', minSupport=0.001, minConfidence=0.01)
model = fpg.fit(df)

Lấy ra các tập phổ biến từ mô hình

In [ ]:
frequent_itemsets = model.freqItemsets
frequent_itemsets.show()

In [ ]:
frequent_itemsets.count()

**Xem chi tiết các itemsets phổ biến**

In [ ]:
frequent_itemsets.select('items').collect()

**Rút ra các bộ luật**

In [ ]:
associate_rules = model.associationRules
associate_rules.show()

In [ ]:
associate_rules.count()

**Xem chi tiết các luật rút ra được**

In [ ]:
associate_rules.select('antecedent','consequent').collect()

**Kết luận** </br>
- Với minSupport = 0.001 ta thu được 4444 tập phổ biến trên tổng số 131209 mẫu. Cho ra tỉ lệ tập phổ biến xấp xỉ 0.34%. </br>
$→$ Dữ liệu không bị lặp nhiều
- Kết hợp tìm các luật với minConfidence = 0.01 ta thu được 5435 luật

**Chạy mô hình trên tập dữ liệu trên với các bộ minSupport và minConfidence khác:**
- minSupport = 0.1
- minConfidence = 0.1

In [ ]:
fpg = fpm.FPGrowth(itemsCol='products', minSupport=0.1, minConfidence=0.1)
model2 = fpg.fit(df)

In [ ]:
model2.freqItemsets.collect()

In [ ]:
model2.associationRules.show()

**Nhận xét:** Khi tăng minSupport lên 0.1, ta chỉ tìm được 2 tập phổ biến là Banana và Bag of Organic Bananas. Với minConfidence = 0.1 thì ta không tìm thấy một luật nào.

**3) Vấn đề về hình thức của các luật tìm thấy và giải pháp khắc phục** </br>
</br>
**Nhận xét:** Có thể thấy các vế của bộ luật chỉ gồm 1 item như: <em>Large Lemons $→$ Limes</em> và thiếu đi các luật mà 2 vế gồm nhiều items.</br>
$→$ Điều cần lưu ý khi sử dụng FPGrowth trong thư viện học máy của PySpark là các luật được rút ra về phía consequent chỉ có 1 item. </br>

**Cách khắc phục:** Sử dụng thư viện **MLxtend** - một trong các thư viện hỗ trợ các công cụ về học máy và khoa học dữ liệu. Dùng **association_rules** để khai thác các luật có minConfience = 0.05 với tập phổ biến ta đã tìm được với minSupport = 0.01

In [ ]:
!pip install mlxtend

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules
import pandas as pd

In [ ]:
frequent_itemsets_df = frequent_itemsets.toPandas()

def get_support(freq,N):
  return freq/N

In [ ]:
frequent_itemsets_temp = pd.DataFrame()
frequent_itemsets_temp['support'] = get_support(frequent_itemsets_df['freq'],df.count())
frequent_itemsets_temp['itemsets'] = frequent_itemsets_df['items']

In [ ]:
new_association_rules = association_rules(frequent_itemsets_temp,metric='confidence',min_threshold=0.01)
new_association_rules

Sau khi tìm lại các luật với minConfidence = 0.01 bởi MLxtend trên tập phổ biến đã tìm được bởi FPGrowth của PySpark với minSupport = 0.001, ta nhận tìm được 6583 luật, nghĩa là có 1148 luật có consequent nhiều hơn 1 items được tìm thấy. </br>
</br>
**Các luật có consequent nhiều hơn 1 item được thể hiện ở DataFrame bên dưới:**

In [ ]:
def get_consequent_size(cons):
  return len(cons)
new_association_rules['length'] = new_association_rules['consequents'].apply(get_consequent_size)
new_association_rules[new_association_rules['length']>1]

# **Bài 2 - Phân lớp**

**1)  Đọc dữ liệu vào Spark và chia dữ liệu thành tập huấn luyện và tập kiểm thử theo tỉ lệ 80:20**


In [ ]:
mushroom_df = spark.read.csv("Lab04-Data/mushrooms.csv", header='True')
mushroom_df.show()

In [ ]:
train, test = mushroom_df.randomSplit(weights=[0.8,0.2], seed=100)
print(train.count(), test.count())

**2) Xây dựng mô hình decision tree trên tập huấn luyện**

In [ ]:
from pyspark.ml import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import OneHotEncoder

In [ ]:
input_col = ['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat']

In [ ]:
# Sử dụng One Hot Encoder để chuyển đổi dữ liệu dạng Categories về dạng Index và biến stages dạng list để lưu thứ tự thực thi với pipeline
stages = []
for col in input_col:
    stringIndexer = StringIndexer(inputCol = col, outputCol = col + 'Index')
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[col + "classVec"])
    stages += [stringIndexer, encoder]

# Tạo vector features là các thuộc tính có trên tập dữ liệu (trừ thuọc tính class) và gom dữ liệu các thuộc tính thành vector
assemblerInputs = [col + "classVec" for col in input_col]
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

#Tạo thuộc tính dạng index cho thuộc tính features vector theo tham số của mô hình Decision Tree yêu cầu
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=2)

# Thêm thuộc tính label dạng numberic từ thuộc tính class dạng categories
labelIndexer = StringIndexer(inputCol="class", outputCol="indexedLabel")

# Thêm vào pipeline
stages += [assembler, labelIndexer, featureIndexer]

In [ ]:
# Khởi tạo mô hình Decision Tree với indexedLabel và indexedFeatures
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")
stages += [dt]

pipeline = Pipeline(stages=stages)

# Training với pipeline
model = pipeline.fit(train)

# Chạy DT trên tập test
predictions = model.transform(test)

predictions.select("prediction", "indexedLabel", "features").show(5)

# Đánh giá mô hình
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(predictions)

y_true = predictions.select("indexedLabel").rdd.flatMap(lambda x: x).collect()
y_pred = predictions.select("prediction").rdd.flatMap(lambda x: x).collect()

confusionmatrix = confusion_matrix(y_true, y_pred)

precision = precision_score(y_true, y_pred, average='micro')

recall = recall_score(y_true, y_pred, average='micro')

treeModel = model.stages[-1]
print(treeModel)
print("Test Accuracy = %g" % (accuracy))
print("Test Error = %g" % (1.0 - accuracy))

print("The Confusion Matrix:\n" + str(confusionmatrix))

print("Precision score: " + str(precision))

print("Recall score: " + str(recall))

Nhận xét:
- Accuracy của mô hình mạng lại khá cao: 0.998141
- Dựa vào confusion matrix chỉ có 3/1614 mẫu được dự đoán sai

**3) Xây dựng mô hình Random Forest trên tập huấn luyện**

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
stages1 = []
for col in input_col:
    stringIndexer = StringIndexer(inputCol = col, outputCol = col + 'Index')
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[col + "classVec"])
    stages1 += [stringIndexer, encoder]

assemblerInputs = [c + "classVec" for c in input_col]
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=2)
labelIndexer = StringIndexer(inputCol="class", outputCol="indexedLabel")

stages1 += [assembler, labelIndexer, featureIndexer]

In [ ]:
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)
stages1 += [rf]

pipeline = Pipeline(stages=stages1)

model = pipeline.fit(train)

predictions = model.transform(test)
  
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(predictions)
y_true = predictions.select("indexedLabel").rdd.flatMap(lambda x: x).collect()
y_pred = predictions.select("prediction").rdd.flatMap(lambda x: x).collect()
confusionmatrix = confusion_matrix(y_true, y_pred)

precision = precision_score(y_true, y_pred, average='micro')

recall = recall_score(y_true, y_pred, average='micro')

print("Test Accuracy = %g" % (accuracy))
print("Test Error = %g" % (1.0 - accuracy))

print("Confusion Matrix:\n" + str(confusionmatrix))

print("Precision score: " + str(precision))

print("Recall score: " + str(recall))

Nhận xét:
- Accuracy của mô hình khá cao: 0.9758
- Có 39/1614 mẫu được dự đoán sai trong tập test

**4) Đánh giá hai mô hình trên tập kiểm thử**


<em>Decision Tree</em>:</br>
    - Thuật toán có độ chính xác 99.8% khi thử nghiệm trên tập test và có tỉ lệ lỗi là 0.2%</br>
<em>RandomForest</em>:</br>
    - Thuật toán có độ chính xác 98.5% khi thử nghiệm trên tập test và có tỉ lệ lỗi là 2.42%</br>
$→$ Đối với tập dữ liệu này, thuật toán Decision Tree cho độ chính xác cao hơn RandomForest nhưng cũng không đáng kể.

# **Bài 3 - Gom cụm**

In [ ]:
!unrar x "/content/Lab04-Data/plants.rar" 

**1) Tiền xử lý dữ liệu**

In [ ]:
plants_df = spark.read.text("plants.data")

In [ ]:
plants_df.show()

In [ ]:
plants_list = plants_df.select('value').rdd.flatMap(lambda x: x).collect()

In [ ]:
plants = []
positions = []

for line in plants_list:
  words = line.split(',')
  plants.append(words[0])
  positions.append(words[1:])

In [ ]:
position_list = []
for sample in positions:
  for p in sample:
    if p in position_list:
      continue
    else:
      position_list.append(p)

In [ ]:
import pandas as pd
df = pd.DataFrame()
df['plant'] = pd.DataFrame(plants)
for pos in position_list:
  pos_col = []
  for sample in positions:
    if pos in sample:
      pos_col.append(1)
    else:
      pos_col.append(0)
  df[pos] = pd.DataFrame(pos_col)

In [ ]:
plants_df = spark.createDataFrame(df)

In [ ]:
plants_df.show()

**2) Thực hiện gom cụm các loài thực vật theo vùng địa lý bằng giải thuật k-means và đánh giá kết quả gom cụm bằng ClusteringEvaluator. Thử nghiệm với một số giá trị k**

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql import SparkSession

In [ ]:
vecAssembler = VectorAssembler(inputCols=position_list, outputCol="features")
new_df = vecAssembler.transform(plants_df)

In [ ]:
silhouette_scores=[]
evaluator = ClusteringEvaluator(featuresCol='features', \
metricName='silhouette', distanceMeasure='squaredEuclidean')

for K in range(2,20):
    KMeans_=KMeans(featuresCol='features', k=K)
    KMeans_fit=KMeans_.fit(new_df)
    KMeans_transform=KMeans_fit.transform(new_df) 
    evaluation_score=evaluator.evaluate(KMeans_transform)
    silhouette_scores.append(evaluation_score)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,1, figsize =(20,8))
ax.plot(range(2,20),silhouette_scores)
ax.set_xlabel('Number of clusters')
ax.set_ylabel('Silhouette Score')

**Nhận xét (ClusteringEvaluator):**</br>
Ở lần thực thi hiện tại, biểu đồ Clustering Evaluator cho thấy các giá trị $k$ (số cluster) phù hợp với tập dữ liệu này có Silhouette score cao nhất là $k=2,4,6,7,8$ (xếp theo thứ tự giảm dần)</br>
Silhouette score: là giá trị tổng thể cho biết các điểm dữ liệu được phân bố vào đúng cụm của nó

**Clustering với $k=2$**

In [ ]:
kmeans = KMeans(k=2, seed=1)
model = kmeans.fit(new_df.select('features'))
transformed = model.transform(new_df)
cluster = transformed.select('plant','prediction')

Thống kê số lượng mẫu thuộc các cluster

In [ ]:
cluster.groupby('prediction').count().orderBy('prediction').show()

**Clustering với $k=4$**

In [ ]:
kmeans = KMeans(k=4, seed=1)
model = kmeans.fit(new_df.select('features'))
transformed = model.transform(new_df)
cluster = transformed.select('plant','prediction')

Thống kê số lượng mẫu thuộc các cluster

In [ ]:
cluster.groupby('prediction').count().orderBy('prediction').show()

# Hết